In [2]:
## import 정보

from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import datetime
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
import time
import re


#### 셀레니움 안쓰고 오로지 bs, requests 를 사용해서
## 네이버가 bs 못쓰게 막아버림 ㅋ ajax와 쿠키의 미친콜라보

In [7]:
## BS 옵션

headers = {"User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
                        AppleWebKit/537.36 (KHTML, like Gecko) \
                        Chrome/91.0.4472.101 Safari/537.36'}


## 기타 옵션

pd.set_option('display.unicode.east_asian_width', True)
re_title = re.compile('[^가-힣|a-z|A-Z ]')   # 정규 표현식

##사이트 옵션 & 카테고리 정보 $  DF 설정

url = ''
columns = [titles, writers, reviews, category1, category2, category3]



100번부터 340번까지 25가지 카테고리.
https://book.naver.com/category/index.naver?cate_code=350
뒤에 3자리씩 붙이는 방식으로 책 카테고리 세분화

https://book.naver.com/category/index.naver?cate_code=100010010&tab=top100&list_type=list&sort_type=publishday&page=1
한번 받아볼까

<a class="N=a:bta.title docs-creator" 
href="http://book.naver.com/bookdb/book_detail.naver?bid=13993822">알려지지 않은 예술가의 눈물과 자이툰 파스타</a>

파란색 글자 태그 따기


In [49]:
url= 'https://book.naver.com/category/index.naver?cate_code=100'
html= requests.get(url)
resp= BeautifulSoup(html.text, 'html.parser')
print(resp)

<!DOCTYPE HTML>

<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title>
소설홈, 소설 : 네이버 책</title>
<meta content="article" property="og:type"/>
<meta content="네이버 책" property="og:title"/>
<meta content="http://book.naver.com/category/index.naver?cate_code=100" property="og:url"/>
<meta content="https://ssl.pstatic.net/static/m/book/icons/book_og_270x270.png" property="og:image"/>
<meta content="책으로 만나는 새로운 세상" property="og:description"/>
<meta content="" property="og:article:thumbnailUrl"/>
<meta content="네이버 책" property="og:article:author"/>
<meta content="https://book.naver.com" property="og:article:author:url"/>
<link href="https://www.naver.com/favicon.ico?book" rel="shortcut icon" type="image/x-icon"/>
<script type="text/javascript">
var __bLogin = false;
var __bAdult = false;
</script>
<link href="/css/release/base_20211207091736.css" rel="stylesheet" type="text/css"/>
<script charse

resp 확인. columns = [titles, writers, url_list] 

In [9]:
url_list = []
temp = ''

for i in range(1,6):
    url = 'https://book.naver.com/category/index.naver?cate_code=100010010&tab=top100&list_type=list&sort_type=publishday&page={}'.format(i)
    html = requests.get(url, headers = headers)
    resp = BeautifulSoup(html.text , 'html.parser')
    resp.findAll('',{'class' : "N=a:bta.title docs-creator" })
    temp = resp.attrs['href']
    url_list.append(temp)
print(url_list)

KeyError: 'href'

In [21]:
url = 'https://book.naver.com/category/index.naver?cate_code=100010010&tab=top100&list_type=list&sort_type=publishday&page={}'.format(i)
html = requests.get(url, headers = headers)
resp = BeautifulSoup(html.text , 'html.parser')
resp.findAll('a')


[<a href="#" name="start">본문시작</a>,
 <a href="/index.naver" id="home_navi">책 홈</a>,
 <a href="#" id="category_location1_depth" onclick="showLocationLayer('location1_depth');return false;" onmouseout="hideLocationLayer('location1_depth')">소설</a>,
 <a href="#" id="category_location2_depth" onclick="showLocationLayer('location2_depth');return false;" onmouseout="hideLocationLayer('location2_depth')">나라별 소설</a>,
 <a href="#" id="category_location3_depth" onclick="showLocationLayer('location3_depth');return false;" onmouseout="hideLocationLayer('location3_depth')">한국소설</a>,
 <a class="N=a:bil.category1,i:100" href="http://book.naver.com/category/index.naver?cate_code=100">소설</a>,
 <a class="N=a:bil.category1,i:110" href="http://book.naver.com/category/index.naver?cate_code=110">시/에세이</a>,
 <a class="N=a:bil.category1,i:160" href="http://book.naver.com/category/index.naver?cate_code=160">경제/경영</a>,
 <a class="N=a:bil.category1,i:170" href="http://book.naver.com/category/index.naver?cate_code

In [34]:
url = 'https://book.naver.com/category/index.naver?cate_code=100010010&tab=top100&list_type=list&sort_type=publishday&page={}'.format(i)
html = requests.get(url, headers = headers)
resp = BeautifulSoup(html.text , 'html.parser')
s = resp.findAll('a', class_ = "N=a:bta.title")
for i in s:
    url_list.append(i.attrs['href'])
    

## 12개밖에 안쏴주네?

In [35]:
url_list

['http://book.naver.com/bookdb/book_detail.naver?bid=13314689',
 'http://book.naver.com/bookdb/book_detail.naver?bid=15512482',
 'http://book.naver.com/bookdb/book_detail.naver?bid=15476484',
 'http://book.naver.com/bookdb/book_detail.naver?bid=15475909',
 'http://book.naver.com/bookdb/book_detail.naver?bid=15475641',
 'http://book.naver.com/bookdb/book_detail.naver?bid=15509935',
 'http://book.naver.com/bookdb/book_detail.naver?bid=15432263',
 'http://book.naver.com/bookdb/book_detail.naver?bid=15369087',
 'http://book.naver.com/bookdb/book_detail.naver?bid=15372732',
 'http://book.naver.com/bookdb/book_detail.naver?bid=15372716',
 'http://book.naver.com/bookdb/book_detail.naver?bid=15372717',
 'http://book.naver.com/bookdb/book_detail.naver?bid=15133927']

In [40]:
url = 'https://book.naver.com/category/index.naver?cate_code=100010010&tab=top100&list_type=list&sort_type=publishday&page=1'
html = requests.get(url, headers = headers)
resp = BeautifulSoup(html.text , 'html.parser')
resp

## bs 로는 막힘 아약스가 막아버림

<script src="/js/jquery-1.9.1.min.js?20211207091736" type="text/javascript"></script>
<script src="/js/logeye.js" type="text/javascript"></script>
<script type="text/javascript">
var ccsrv = "cc.naver.com";
var nclk_evt = 3;

nclk_do();

function emptyImg(obj) {
	obj.src = "https://ssl.pstatic.net/static/book/image/noimg4.gif";
	obj.width = "106";
	obj.height = "154";
}

function callEyelogApi(storeId) {
	if(!storeId) {
		return ;
	}

	var obj = {
		'storeId':storeId
	}

	logeye.call.setParams(obj);
	logeye.call.callLogEye();
}
</script>
<div id="content">
<div class="blind" id="start"><a href="#" name="start">본문시작</a></div>
<div class="location">
<h2 class="blind">한국소설</h2>
<ul class="navi">
<li class="home"><a href="/index.naver" id="home_navi">책 홈</a></li>
<li>&gt;</li>
<li class="select">
<a href="#" id="category_location1_depth" onclick="showLocationLayer('location1_depth');return false;" onmouseout="hideLocationLayer('location1_depth')">소설</a>
</li>
<li>&gt;</li>
<li class="selec

# 여기서부터는 셀레니움 코드

/html/body/div/div[3]/div[2]/div[9]/ol/li[1]/dl/dt/a
/html/body/div/div[3]/div[2]/div[9]/ol/li[2]/dl/dt/a
/html/body/div/div[3]/div[2]/div[9]/ol/li[20]/dl/dt/a

In [25]:
# 셀레니움 옵션.

options = webdriver.ChromeOptions()
# options.add_argument('headless')
options.add_argument('window-size=1920x1080')
# options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
options.add_argument('lang=ko_KR')
options.add_argument('disable_gpu')
options.add_argument('--user-agent=Mozilla/5.0 (iPhone; CPU iPhone OS 10_3 like Mac OS X) AppleWebKit/602.1.50 (KHTML, like Gecko) CriOS/56.0.2924.75 Mobile/14E5239e Safari/602.1')
driver = webdriver.Chrome('./chromedriver', options=options)

C:\Users\Nokdu\.conda\envs\ai_exam\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  # Remove the CWD from sys.path while we load stuff.


In [50]:
temp = None
url_list=[]

In [ ]:
for x in range(10,34):
    for y in range(1,20):
        try:
            for i in range(1,6):
                url = 'https://book.naver.com/category/index.naver?cate_code={}0{}0&tab=top100&list_type=list&sort_type=publishday&page={}'.format(x,y,i)
                driver.get(url)
                for j in range(1,21):
                    temp = driver.find_element_by_xpath('/html/body/div/div[3]/div[2]/div[9]/ol/li[{}]/dl/dt/a'.format(j))
                    new_url = temp.get_attribute("href")
                    url_list.append(new_url)
        except:
            print()
            print(url_list)

In [1]:
## import 정보

from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import datetime
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
import time
import re

## 기타 옵션

pd.set_option('display.unicode.east_asian_width', True)
re_title = re.compile('[^가-힣|a-z|A-Z ]')   # 정규 표현식

# 셀레니움 옵션.

options = webdriver.ChromeOptions()
# options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
options.add_argument('lang=ko_KR')
options.add_argument('disable_gpu')
driver = webdriver.Chrome('./chromedriver', options=options)

## 변수 선언

url_list = []

df = pd.DataFrame()

## review xpath

# /html/body/div/div[3]/div[6]/div/ul/li[1]/dl/dt/a
# /html/body/div/div[3]/div[6]/div/ul/li[2]/dl/dt/a
# /html/body/div/div[3]/div[6]/div/ul/li[10]/dl/dt/a
# /html/body/div/div[3]/div[6]/div/ul/li[9]/dl/dt/a

# 리뷰페이지수
# /html/body/div/div[3]/div[6]/div/div[2]/a[1]
# /html/body/div/div[3]/div[6]/div/div[2]/a[2]
# /html/body/div/div[3]/div[6]/div/div[2]/a[3]

for x in range(10, 12):
    #10부터 34까지 있고 중간에 없는것도 있음
    for y in range(1, 20):
        ## 100개 마다
        for i in range(1,6):
            try:
                url = 'https://book.naver.com/category/index.naver?cate_code={}0{}0&tab=top100&list_type=list&sort_type=publishday&page={}'.format(x,y,i)
                driver.get(url)
                for j in range(1,21):
                    temp = driver.find_element_by_xpath('/html/body/div/div[3]/div[2]/div[9]/ol/li[{}]/dl/dt/a'.format(j))
                    new_url = temp.get_attribute("href")
                    url_list.append(new_url)
            except:
                print('.', end='')
                break
                # print(len(url_list))
        try:
            for z in range(100):
                driver.get(url_list[z])
                titles = []
                writers = []
                reviews = []
                title = driver.find_element_by_xpath('/html/body/div[1]/div[3]/div[4]/div[1]/h2/a').text
                writer = driver.find_element_by_xpath('//*[@id="container"]/div[4]/div[1]/div[2]/div[2]/a[1]').text
                #리뷰 페이지 클릭
                #1개의 리뷰
                try:
                    for a in range(2,4):
                        ## 리뷰 30개만.
                        for b in range(1, 11):
                            driver.get(url_list[z].replace('book_detail', 'review'))
                            driver.find_element_by_xpath('/html/body/div/div[3]/div[6]/div/ul/li[{}]/dl/dt/a'.format(b)).click
                            time.sleep(0.5)
                            review = driver.find_element_by_xpath('/html/body/div[6]/div[1]/div[2]/div[2]/div[2]/div[2]/div/div/div[8]/div[1]/div/table[2]/tbody/tr/td[2]/div[1]/div/div/div[3]').text
                            titles.append(title)
                            reviews.append(review)
                            writers.append(writer)

                except:
                    print('.')  #print('리뷰가 없습니다.')
            df_review_100 = pd.DataFrame({'titles': titles, 'writers': writers, 'reviews':reviews})
            df_review_100.to_csv('./datasets/review_{}_{}_cate'.format(x,y))
        except:
            print('이 카테고리는 없습니다.')




C:\Users\Nokdu\.conda\envs\ai_exam\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
C:\Users\Nokdu\.conda\envs\ai_exam\lib\site-packages\ipykernel_launcher.py:55: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


.이 카테고리는 없습니다.
.이 카테고리는 없습니다.
.이 카테고리는 없습니다.
.이 카테고리는 없습니다.
.이 카테고리는 없습니다.
.이 카테고리는 없습니다.
.이 카테고리는 없습니다.
.이 카테고리는 없습니다.
.이 카테고리는 없습니다.
.이 카테고리는 없습니다.
.이 카테고리는 없습니다.
.이 카테고리는 없습니다.
.이 카테고리는 없습니다.
.이 카테고리는 없습니다.
.이 카테고리는 없습니다.
.이 카테고리는 없습니다.
.이 카테고리는 없습니다.
.이 카테고리는 없습니다.
.이 카테고리는 없습니다.
.이 카테고리는 없습니다.
.이 카테고리는 없습니다.
.이 카테고리는 없습니다.
.이 카테고리는 없습니다.
.이 카테고리는 없습니다.
.이 카테고리는 없습니다.
.이 카테고리는 없습니다.
.이 카테고리는 없습니다.
.이 카테고리는 없습니다.


C:\Users\Nokdu\.conda\envs\ai_exam\lib\site-packages\ipykernel_launcher.py:68: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
C:\Users\Nokdu\.conda\envs\ai_exam\lib\site-packages\ipykernel_launcher.py:69: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
C:\Users\Nokdu\.conda\envs\ai_exam\lib\site-packages\ipykernel_launcher.py:77: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
C:\Users\Nokdu\.conda\envs\ai_exam\lib\site-packages\ipykernel_launcher.py:79: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


.
.
.
.
.
.
.
.
이 카테고리는 없습니다.
.이 카테고리는 없습니다.
.이 카테고리는 없습니다.
.이 카테고리는 없습니다.
.이 카테고리는 없습니다.
.이 카테고리는 없습니다.
.이 카테고리는 없습니다.
.이 카테고리는 없습니다.
.이 카테고리는 없습니다.
.이 카테고리는 없습니다.


In [27]:
driver = webdriver.Chrome('./chromedriver', options=options)
driver.get('https://blog.naver.com/wlstjs0808/222530977454')
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
print(soup)

C:\Users\Nokdu\.conda\envs\ai_exam\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.


<html data-useragent="Mozilla/5.0 (iPhone; CPU iPhone OS 10_3 like Mac OS X) AppleWebKit/602.1.50 (KHTML, like Gecko) CriOS/56.0.2924.75 Mobile/14E5239e Safari/602.1" lang="ko"><head prefix="og: http://ogp.me/ns# article: http://ogp.me/ns/article#">
<meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
<meta content="width=device-width,initial-scale=1.0,maximum-scale=1.0,minimum-scale=1.0,user-scalable=no,target-densitydpi=medium-dpi,viewport-fit=cover" name="viewport"/>
<meta content="index,follow" name="robots"/>
<link href="/favicon_192x192.png?1" rel="icon" type="image/png"/>
<link href="/blog_web_iOS_iPhone@3x_180X180.png" rel="apple-touch-icon-precomposed" sizes="180x180"/>
<link href="/blog_web_iOS_iPadPro@2x_167X167.png" rel="apple-touch-icon-precomposed" sizes="167x167"/>
<link href="/blog_web_iOS_iPad@2x_152X152.png" rel="apple-touch-icon-precomposed" sizes="152x152"/>
<link href="/blog_web_iOS_iPhone@2x_120X120.png" rel="apple-touch-icon-precomposed"/>
<link c

In [19]:
a = soup.findAll( class_="se-main-container")
print(a)

[<div class="se-main-container">
<div class="se-component se-image se-l-default" id="SE-892410af-10e6-4098-b316-64e622121e73">
<div class="se-component-content se-component-content-normal">
<div class="se-section se-section-image se-l-default se-section-align-" style="max-width:400px;">
<div class="se-module se-module-image" style="">
<a class="se-module-image-link __se_image_link __se_link" data-linkdata='{"id" : "SE-892410af-10e6-4098-b316-64e622121e73", "src" : "https://mblogthumb-phinf.pstatic.net/MjAyMTEwMDhfMTY2/MDAxNjMzNjgzOTMxNzQy.i0iV05I87DuoQtkSEvkBafWZnYT7YRNGoS9Zo-txdYUg.b5p7rK1lHuspxsKdA__oxuLw4gBOymvYMJgQto9VKV0g.JPEG.wlstjs0808/SE-892410af-10e6-4098-b316-64e622121e73.jpg", "originalWidth" : "400", "originalHeight" : "400", "linkUse" : "false", "link" : ""}' data-linktype="img" href="#" onclick="return false;" style="">
<img alt="" class="se-image-resource _lazy-loading-target-image" data-height="400" data-lazy-src="" data-top="285" data-width="400" id="img_2" src="https:

In [22]:
b = ''
c = ''
for i in a:
    c = i.get_text()
    c = re.compile('[^가-힣 ]').sub(' ', c)
    b += c

In [23]:
print(b)

                          취업  해고  구조조정  자영업  재건축   한국에서 먹고사는 문제의 고단함과 쓸쓸함을 지적이고 균형 잡힌 시선으로 포착하는   편의 연작소설          구입한지는 꽤 오래된 책인데 차일피일 미뤄두다가 다른 책을 먼저 읽는 바람에  이제서야 완독하게 되었다계간지를 읽을 때나 다른 책을 읽을 때 가끔 이 책이 언급되어 궁금했었다몰랐는데 작가님이 유명하신 분이시다  년 동안  동아일보 에서 근무한 기자로 장편소설  표백 으로 한겨레 문학상을 받으며 작품활동을 시작했고 그 밖에도 여러가지 책을 쓰셨다다른 책도 읽어보고 싶어지는 걸    목차    부 자르기알바생 자르기대기발령공장 밖에서  부 싸우기현수동 빵집 삼국지사람 사는 집카메라 테스트대외 활동의 신  부 버티기모두  친절하다음악의 가격새들은 나는 게 재미있을까 목차를 보아만봐도 내용이 심상치 않다특히나 많이 언급되었던  공장 밖에서 는 쌍용자동차 정리해고를 모티프로 한 소설이었는데 투쟁의 모습이 고스란히 느껴져 매우 실감났다 진짜 구호도 있었다 해고는 살인이다  라는 문구가 적힌 현수막이 도장 공장 옥상에 걸렸다해고는 살인이었으므로 그들은  죽은 자 들이었고  해고자 명단에 오르지 않은 사람은  산 자 가 되었다  공장 밖에서         이 책을 읽고나니 친구가 빌려준 책이 있었다마지막 파트만 읽으면되서 주말에 포스팅할 것 같은데  사회역학을 다룬  아픔이 길이 되려면 이라는 책이다신기하게도 두 책은 각각의 연관성이 있었다 산 자들 이 한국의 노동 문제를 주로 다루고 있다면  아픔이 길이 되려면 은 한국사회 노동자들의 건강 책임을 개인적인 이유보다  사회적 책임 에 중점을 두고 있다결과와 원인이라고 해야할까  원인의 원인이라고 해야할까 두 책을 읽고 마음이 무거워졌다나는 나이가 어려서 한번도 정리해고나 파업에 대해 걱정해본적이 없다미혼 여성인 내가 이러한 일들을      공감한다고 하면 거짓말이겠지만  다시는 되풀이 되지 않았으면 하는 마음이 든다나의 아